# Prerequisites

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import functools, os, re, shutil, string, sys
from collections import Counter
from datetime import datetime
from IPython.display import display, HTML
from pathlib import Path

import nltk, pyLDAvis, spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scattertext as st
import tensorflow as tf
import tomotopy as tp
import xgboost as xgb

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from scipy.stats import pointbiserialr
from sklearn import linear_model, naive_bayes, svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, matthews_corrcoef
from sklearn.model_selection import GridSearchCV
from tqdm.notebook import tqdm

from dict_analysis import read_dictionary, get_count

nltk_dir = Path.cwd()/"models"
assert nltk_dir.exists()
os.environ["NLTK_DATA"] = str(nltk_dir)
os.environ['OMP_NUM_THREADS'] = '16'

pyLDAvis.enable_notebook()

print("\nReady for next step...")

## Getting Started

In [ ]:
if not tf.config.list_physical_devices("GPU"):
    print("No GPU detected... You probably want to restart with a GPU-enabled runtime")
else:
    print("Found a GPU! Ready for next step...")

If you see a message above stating that no GPU is detected, make sure you are running this from an interactive GPU session

## Loading the IMDB dataset

For simplicity, we're going to use a large publicly available dataset to demonstrate machine learning using multiple techniques. This dataset has already been loaded for you in the setup process.

In [ ]:
texts_path = Path.cwd() / "texts"
dicts_path = Path.cwd() / "dictionaries"
dataset_dir = texts_path / 'aclImdb'
train_dir = dataset_dir / 'train'
test_dir = dataset_dir / 'test'

# Generate Samples for the NN
batch_size = 32
seed = 24601

print(f"\n====Loading Training Dataset==== - {datetime.now()}", flush=True)
raw_train = tf.keras.utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)
class_names = raw_train.class_names
train_ds = raw_train.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

print(f"\n====Loading Validation Dataset==== - {datetime.now()}", flush=True)
raw_validation = tf.keras.utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)
val_ds = raw_validation.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

print(f"\n====Loading Test Dataset==== - {datetime.now()}", flush=True)
raw_test = tf.keras.preprocessing.text_dataset_from_directory(test_dir, batch_size=batch_size)
test_ds = raw_test.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

print("\nReady for next step...")

In [ ]:
# Convert training sample to pandas for non-tensorflow processing (i.e., traditional ML)
print(f"\n====Converting Training Sample to Pandas==== - {datetime.now()}", flush=True)
reviews = []
sentiments = []
for text, sentiment in train_ds.as_numpy_iterator():
  reviews.extend(text)
  sentiments.extend(sentiment)

full_train_data = pd.DataFrame(zip(reviews,sentiments), columns=['review','sentiment'])

# Convert training sample to pandas for non-tensorflow processing (i.e., traditional ML)
print(f"\n====Converting Validation Sample to Pandas==== - {datetime.now()}", flush=True)
reviews = []
sentiments = []
for text, sentiment in val_ds.as_numpy_iterator():
  reviews.extend(text)
  sentiments.extend(sentiment)

full_validation_data = pd.DataFrame(zip(reviews,sentiments), columns=['review','sentiment'])

# Convert test sample to pandas for non-tensorflow processing (e.g., traditional ML, dictionary-based coding)
print(f"\n====Converting Test Sample to Pandas==== - {datetime.now()}", flush=True)
reviews = []
sentiments = []
for text, sentiment in test_ds.as_numpy_iterator():
  reviews.extend(text)
  sentiments.extend(sentiment)

full_test_data = pd.DataFrame(zip(reviews,sentiments), columns=['review','sentiment'])

print(f"\n====Generating 3000 Row Test Dataset Subsample (for demonstration time-saving purposes)==== - {datetime.now()}", flush=True)
np.random.seed(seed)
test_data = full_test_data.sample(n=3000)
test_data['txt_sent'] = test_data.apply(lambda x: "Positive" if x["sentiment"]==1 else "Negative", axis=1)
test_data['review'] = test_data['review'].apply(lambda x: x.decode("utf-8").replace("\\", "").replace("<br />", " "))

# Stopwords setup
print(f"\n====Getting Stopwords Setup==== - {datetime.now()}", flush=True)
stops = nltk.corpus.stopwords.words('english')+["'s", '&']

print("\nReady for next step...")

In [ ]:
# Load the dictionaries
dictionaries = {}
for file in dicts_path.glob(f'*.dict'):
  dictionary_data = read_dictionary(file)
  dictionaries[dictionary_data['var_name']] = dictionary_data

#Spin up preprocessing pipeline
print("Preprocessing texts - this may take a while")
nlp = spacy.load("en_core_web_sm")
docs = nlp.pipe(test_data['review'].tolist())
del nlp

# Update the pandas dataframe with the tokenized reviews
preprocessed = []
for doc in docs:
    preprocessed.append(
        [token.text.lower()
         for token in doc
         if token.pos_ not in ["PUNCT", "SYM", "NUM", 'X']
         and token.text.lower() not in stops
        ]
    )
test_data['review_tokens'] = preprocessed

print("\nReady for next step...")

# Rules-based analyses as a point for comparison
---

This data set links IMDB movie reviews to the overall 'sentiment' (whether it's a positive review or negative review) of the movie. Let's take a look at a few of the reviews

In [ ]:
# Get one batch from the training set and print first five reviews
for _, row  in test_data.head().iterrows():
    print(f"Review: {row['review'][:130]}...")
    print(f"Sentiment: {row['txt_sent']}\n")

print("\nReady for next step...")

In dictionary-based coding, we look at the frequency with which a list of words thought to be associated with a construct (in this case, positive/negative sentiment) appear in a corpus of texts.

The selection of words in the dictionaries is *crucial* for measure validity and reliability. In this case, we're looking at positive/negative words from Henry (2008). Let's take a look at these dictionaries:

In [ ]:
# Print out all dictionaries' names, titles, and word lists.
for name, dictionary in dictionaries.items():
    print(f"\nNAME: {name}")
    print(f"TITLE: {dictionary['title']}")
    print(f"WORDS: {', '.join(sorted(dictionary['words']))}\n{'-'*20}")

print("\nReady for next step...")

These words were selected for their relevance in business communications. Yet our texts are movie reviews. We should probably see whether these dictionaries still make sense.

Let's conduct a *concordance* analysis or a Key Word In Context (KWIC) analysis to see if these words have face validity: (Remember that stop words have been removed, so the english won't flow perfectly here...)

In [ ]:
# Looks for instances of the 'look_for' word in the corpus and prints their contexts.
look_for = "beat"
nltk.Text([word for id, row in test_data.iterrows() for word in row['review_tokens']]).concordance(look_for)# Counts the number of instances of 'wordlist' words in 'tokens'

print("\nReady for next step...")

Chances are good that you found some instances that didn't seem quite right.

**Why this might be bad:** Language could differ in the current texts than in the texts from which these dictionaries were developed.

**Why this might be OK:** Finding some false-positives is inevitable even with the most appropriate dictionaries. We try to minimize this, but it's a balancing act: the false negatives created by omitting a word that is usually used in-context creates measurement error variance as well.

Let's hold our nose for a moment and assume we're OK with the current dictionaries. Let's conduct the actual dictionary-based computer-aided text analysis:

In [ ]:
# Conducts and displays Dictionary-based CATA for Henry (2008) positivity and negativity dictionaries
test_data["positivity_henry_08"] = test_data["review_tokens"].apply(lambda x: get_count(x, dictionaries['Tone_Positivity_Henry08']['words']))
test_data["negativity_henry_08"] = test_data["review_tokens"].apply(lambda x: get_count(x, dictionaries['Tone_Negativity_Henry08']['words']))
test_data.head()

Now that we have positivity and negativity scores, we can calculate a point-biserial correlation with the actual sentiment to see how accurate we are...

In [ ]:
# Calculates and displays point-biserial correlations for positive and negative CATA with ground-truth sentiment
pos_pbsr = pointbiserialr(x=test_data['sentiment'], y=test_data['positivity_henry_08'])
neg_pbsr = pointbiserialr(x=test_data['sentiment'], y=test_data['negativity_henry_08'])

print(f"The correlation between sentiment and the positivity dictionary is {pos_pbsr[0]:.02}; p = {pos_pbsr[1]:.03}")
print(f"The correlation between sentiment and the negativity dictionary is {neg_pbsr[0]:.02}; p = {neg_pbsr[1]:.03}")
print("\nReady for next step...")

However, here we have two measures of sentiment rather than one. There are many ways the literature has combined such measures into a single sentiment score. A couple common approaches are:
* Difference scores
* Janis-Fadner coefficient of imbalance

Let's look at how these overall sentiment scores correlate:

In [ ]:
# Returns the Janis-Fadner Coefficient of Imbalance as a function of positive, negative, and total words
def coeff_of_imbalance(pos, neg, tot_words):
  if pos == neg:
    return 0
  elif pos > neg:
    return ((pos**2 - pos*neg))/((pos+neg)*tot_words)
  elif pos < neg:
    return ((neg**2 - pos*neg))/((pos+neg)*tot_words)


# Adds sentiment columns for both difference scores and J-F coefficients
test_data["sentiment_henry_08"] = test_data["positivity_henry_08"] - test_data["negativity_henry_08"]
test_data["coeff_imb_henry_08"] = test_data.apply(lambda row: coeff_of_imbalance(row.positivity_henry_08, row.negativity_henry_08, len(row.review_tokens)), axis=1)


# Calculates and displays point-biserial correlations for difference scores and J-F coefficients with ground-truth sentiment
sent_pbsr = pointbiserialr(x=test_data['sentiment'], y=test_data['sentiment_henry_08'])
coi_pbsr = pointbiserialr(x=test_data['sentiment'], y=test_data['coeff_imb_henry_08'])

print(f"The correlation between sentiment and the difference score is          {sent_pbsr[0]:.02}; p = {sent_pbsr[1]:.03}")
print(f"The correlation between sentiment and the coefficient of imbalance is  {coi_pbsr[0]:.02}; p = {coi_pbsr[1]:.03}")
print("\nReady for next step...")

Chances are these word lists need refining... let's use the `scattertext` package to examine the distribution of words over positive/negative sentiment texts:

In [ ]:
# Create and display a scattertext explorer plot
st_corpus = st.CorpusFromPandas(test_data, category_col='txt_sent', text_col='review').build().compact(st.AssociationCompactor(2000))
st_html = st.produce_scattertext_explorer(st_corpus,
                                          category='Positive',
                                          category_name='Positive',
                                          not_categories=['Negative'],
                                          sort_by_dist=False,
                                          term_scorer=st.CredTFIDF(st_corpus),
                                          background_color='#e5e5e3'
                                          )
display(HTML(st_html))

We can use this chart to see words that are:

*   Used frequently in negative reviews and infrequently in positive reviews, but are not listed in our negative word list. (False Negatives)
*   Used frequently in positive reviews and infrequently in negative reviews, but are not listed in our positive word list. (False Negatives)
*   in our word lists, but do not discriminate well between positive and negative reviews. (False Positives)

As a reminder, here is what the Henry (2008) word lists contain:

In [ ]:
# Print out all dictionaries' names, titles, and word lists.
for name, dictionary in dictionaries.items():
    print(f"\nNAME: {name}")
    print(f"TITLE: {dictionary['title']}")
    print(f"WORDS: {', '.join(sorted(dictionary['words']))}\n{'-'*20}")

print("\nReady for next step...")

#### ACTIVITY
---

* Use the scattertext plot to add words to the positive/negative dictionaries based on:
  * How well they discriminate between positive and negative reviews
  * Whether you could theoretically justify their linkage to positive/negative sentiment (e.g., Just because 'Seagal' tends to be in bad movies, doesn't mean we should use his name to influence negative sentiment scores.
* Go through the existing dictionaries and remove words that seem to cause problems in the dictionary-based analysis

In [ ]:
# ~~~~~CHANGE THESE LISTS~~~~~
add_to_positive = ["great", "brilliant", "perfect", "wonderful", "favorite", "loved"]
add_to_negative = ["worst", "terrible", "awful", "waste"]
remove_from_positive = [""]
remove_from_negative = ["declined"]


# Runs the CATA for the custom dictionaries and calculates overall sentiment scores
custom_pos = list(set(dictionaries['Tone_Positivity_Henry08']['words']+add_to_positive)-set(remove_from_positive))
custom_neg = list(set(dictionaries['Tone_Negativity_Henry08']['words']+add_to_negative)-set(remove_from_negative))
test_data["positivity_custom"] = test_data["review_tokens"].apply(lambda x: get_count(x, custom_pos))
test_data["negativity_custom"] = test_data["review_tokens"].apply(lambda x: get_count(x, custom_neg))
test_data["sentiment_custom"] = test_data["positivity_custom"] - test_data["negativity_custom"]
test_data["coeff_imb_custom"] = test_data.apply(lambda row: coeff_of_imbalance(row.positivity_custom, row.negativity_custom, len(row.review_tokens)), axis=1)


# Calculates point-biserial correlations for custom dictionaries with ground-truth sentiment
cus_pos_pbsr = pointbiserialr(x=test_data['sentiment'], y=test_data['positivity_custom'])
cus_neg_pbsr = pointbiserialr(x=test_data['sentiment'], y=test_data['negativity_custom'])
cus_sent_pbsr = pointbiserialr(x=test_data['sentiment'], y=test_data['sentiment_custom'])
cus_coi_pbsr = pointbiserialr(x=test_data['sentiment'], y=test_data['coeff_imb_custom'])


# Displays comparison of Henry (08) and custom dictionaries based on point-biserial corellations
print(f"The correlation between sentiment and the positivity dictionary is......... ORIGINAL: {pos_pbsr[0]:.02}; p = {pos_pbsr[1]:.02} --- CUSTOM: {cus_pos_pbsr[0]:.02}; p = {cus_pos_pbsr[1]:.02}")
print(f"The correlation between sentiment and the negativity dictionary is......... ORIGINAL: {neg_pbsr[0]:.02}; p = {neg_pbsr[1]:.02} --- CUSTOM:{cus_neg_pbsr[0]:.02}; p = {cus_neg_pbsr[1]:.02}")
print(f"The correlation between sentiment and the difference score is.............. ORIGINAL: {sent_pbsr[0]:.02}; p = {sent_pbsr[1]:.02} --- CUSTOM: {cus_sent_pbsr[0]:.02}; p = {cus_sent_pbsr[1]:.02}")
print(f"The correlation between sentiment and the coefficient of imbalance is...... ORIGINAL: {coi_pbsr[0]:.02}; p = {coi_pbsr[1]:.02} --- CUSTOM: {cus_coi_pbsr[0]:.02}; p = {cus_coi_pbsr[1]:.02}")
print("\nReady for next step...")

## VADER (and other weighted rules-based sentiment analyses)
---

VADER is an acronym for "**V**alence **A**ware **D**ictionary and s**E**ntiment **R**easoner" and builds on basic dictionary-based computer-aided text anaylses by [applying weights to sentiment words](https://github.com/cjhutto/vaderSentiment/blob/master/vaderSentiment/vader_lexicon.txt).

For example, the word '*awesome*' has a score of 3.1, whereas '*nice*' has a score of 1.8, and '*horrible*' has a score of -2.5.

The VADER algorithm also explicitly addresses negation (e.g., 'isn't horrible') and boosting (e.g., 'very horrible') in a way that routine dictionary-based approaches do not.

Let's take a look at some examples of what VADER produces for some sample phrases.

In [ ]:
# Have VADER code four sample sentences and display the results
vader_coder = SentimentIntensityAnalyzer()
good_phrase = "This is an awesome movie"
bad_phrase = "This is a horrible movie"
negated_bad = "This isn't a horrible movie"
boosted_bad = "This is a very horrible movie"

print(f"'{good_phrase}' is coded as {vader_coder.polarity_scores(good_phrase)}")
print(f"'{bad_phrase}' is coded as {vader_coder.polarity_scores(bad_phrase)}")
print(f"'{negated_bad}' is coded as {vader_coder.polarity_scores(negated_bad)}")
print(f"'{boosted_bad}' is coded as {vader_coder.polarity_scores(boosted_bad)}")
print("\nReady for next step...")

On face, this seems a significant improvement over what we would see with a generic dictionary-based computer-aided text analysis.

Let's see how well it compares quantitatively on our corpus of IMDB movie reviews:

In [ ]:
# Add a VADER sentiment column to the DataFrame based on the compound score for each text
test_data["vader_comp"] = test_data["review"].apply(lambda x: vader_coder.polarity_scores(x)['compound'])


# Correlate the VADER sentiment with the ground-truth sentiment and display the results (alongside previous results)
vader_pbsr = pointbiserialr(x=test_data['sentiment'], y=test_data['vader_comp'])

print(f"The correlation between sentiment and the VADER score is................... ORIGINAL: {vader_pbsr[0]:.02}; p = {vader_pbsr[1]:.02}")
print(f"{'-'*120}")
print(f"The correlation between sentiment and the difference score is.............. ORIGINAL: {sent_pbsr[0]:.02}; p = {sent_pbsr[1]:.02} --- CUSTOM: {cus_sent_pbsr[0]:.02}; p = {cus_sent_pbsr[1]:.02}")
print(f"The correlation between sentiment and the coefficient of imbalance is...... ORIGINAL: {coi_pbsr[0]:.02}; p = {coi_pbsr[1]:.02} --- CUSTOM: {cus_coi_pbsr[0]:.02}; p = {cus_coi_pbsr[1]:.02}")
print("\nReady for next step...")

Clearly the original Henry (2008) dictionaries didn't fare well against VADER - but that wasn't a fair comparison.

In contrast, if you were thorough in your refinement of the positive/negative dictionaries, you may well have matched or even surpassed the correlation from VADER. VADER was developed using social media texts. So while perhaps closer to movie reviews than the business texts used by Henry, even the more nuanced approach to coding used by VADER may struggle to outperform a dictionary custom-developed to your context.

# Supervised Machine Learning
---

Dictionary-/rules-based approaches are particularly valuable when you do not have a large number of already classified texts. This is often the case in our research, where the reason many business scholars are interested in CATA is because we do not have access to the 'ground truth' for a large number of observations.

However, as ground-truth observations become available, we can use sequence classification algorithms in machine learning to have the computer learn to distinguish between positive and negative sentiment in text.

However, before we can do that, we must first figuratively teach the computer to read.

In [ ]:
# Converts texts into Term Frequency-Inverse Document Frequency (TF-IDF) vectors
vectorizer = TfidfVectorizer(max_features=10000)
x_train_tfidf = vectorizer.fit_transform(full_train_data['review'])
x_validation_tfidf = vectorizer.transform(full_validation_data['review'])
x_test_tfidf = vectorizer.transform(full_test_data['review'])
print("\nReady for next step...")

### Logistic Regression
---

This technique should sound familiar. This workhorse of the statistical world reappears in the machine learning world as a basic model for classification. Here we're regressing the 'ground truth' sentiment on the words used in the text (expressed as a tf-idf vector).

In [ ]:
# ~~~~~HYPERPARAMETERS~~~~~
penalty = "l2"     # penalizes more complex models - options: l2, l1, elasticnet, none
C = 1              # Regularization parameter - smaller numbers --> greater regularization
solver = 'lbfgs'   # optimization algorithm - lbfgs, newton-cg, liblinear, sag, saga
max_iter = 100     # maximum number of iterations - must be an integer


# Train the logistic regression classifier
lr_classifier = linear_model.LogisticRegression(penalty=penalty, C=C, solver=solver, max_iter=max_iter, n_jobs=-1)
lr_classifier.fit(x_train_tfidf,full_train_data['sentiment'])
lr_predictions = lr_classifier.predict(x_test_tfidf)


# Estimate and print the accuracy and phi coefficient for the logistic regression classifier
lr_accuracy = accuracy_score(lr_predictions, full_test_data['sentiment'])
lr_phi = matthews_corrcoef(lr_predictions, full_test_data['sentiment'])

print(f"Logistic Regression accuracy: {lr_accuracy:.2%}")
print(f"Logistic Regression phi coefficient (correlation): {lr_phi:.02}")
print("\nReady for next step...")

### Naïve Bayes
---


Like with logistic regression, you have likely worked with a foundational component of the naïve Bayes classifier in statistics. This classifier uses a key insight from Bayesian statistics to use words to predict a classification. Specifically, we're going to use Bayes' rule to find the P(classification|words) given the P(words|classification), P(classification), and P(words).

In [ ]:
# ~~~~~HYPERPARAMETERS~~~~~
alpha = 1.00       # Additive smoothing parameter - Float
fit_prior = True   # Whether to learn the prior class probabilities from data or use uniform prior


# Train the naive bayes classifier
nb_classifier = naive_bayes.MultinomialNB(alpha=alpha, fit_prior=fit_prior)
nb_classifier.fit(x_train_tfidf,full_train_data['sentiment'])
nb_predictions = nb_classifier.predict(x_test_tfidf)


# Estimate and print the accuracy and phi coefficient for the naive bayes classifier
nb_accuracy = accuracy_score(nb_predictions, full_test_data['sentiment'])
nb_phi = matthews_corrcoef(nb_predictions, full_test_data['sentiment'])

print(f"Naive Bayes' accuracy: {nb_accuracy:.2%}")
print(f"Naive Bayes' phi coefficient (correlation): {nb_phi:.02}")
print("\nReady for next step...")

### Random Forest
---


The random forest classifier is what is called an 'ensemble' classifier. That is, the random forest classifier actually solicits the classifications from a number of other classifiers and treats them as 'votes', tallies the votes and produces a classification on that basis. It's a little more complicated than that, but this is the basic idea. The reason it is called a 'random **forest**'? Well, the classifiers is solicits votes from are 'decision **trees**.'

In [ ]:
# ~~~~~HYPERPARAMETERS~~~~~
n_estimators = 100      # Number of trees in the forest
criterion = 'gini'      # Measure of the quality of a split in the decision tree - gini or entropy
max_depth = None        # Maximum depth the trees in the forest can have - None or integer
min_samples_split = 2   # Minimum number of texts in the branch when a split is made - integer (technically you can have a float, but stick with integer)
min_samples_leaf = 1    # Minimum number of texts in a leaf on the decision tree - integer (technically you can have a float, but stick with integer)


# Train the random forest classifier
rf_classifier = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, max_depth=max_depth,
                                       min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, n_jobs=-1)
rf_classifier.fit(x_train_tfidf,full_train_data['sentiment'])
rf_predictions = rf_classifier.predict(x_test_tfidf)


# Estimate and print the accuracy and phi coefficient for the random forest classifier
rf_accuracy = accuracy_score(rf_predictions, full_test_data['sentiment'])
rf_phi = matthews_corrcoef(rf_predictions, full_test_data['sentiment'])

print(f"Random Forest accuracy: {rf_accuracy:.2%}")
print(f"Random Forest phi coefficient (correlation): {rf_phi:.02}")
print("\nReady for next step...")

### Support Vector Machine
---


The support vector machine has become a very popular classifier for its flexibility. It handles high feature/sample size ratios well, supports several kernel functions for when the decision boundary between two sets are not linearly separable, and is pretty memory efficient for what it does.

It can, however, quite slow... be prepared to wait on this one for a while...

In [ ]:
# ~~~~~HYPERPARAMETERS~~~~~
C = 1.0             # Regularization parameter - smaller numbers --> greater regularization
kernel = 'linear'   # Kernel type to use - linear, poly, rbf, sigmoid
degree = 3          # Degree of polynomial kernel (used if you use 'poly' kernel only)
gamma = 'auto'      # Kernel coefficient for rbf, poly, and sigmoid kernels - auto, scale or a floating point number


# Train the support vector machine classifier
svm_classifier = svm.SVC(C=C, kernel=kernel, degree=degree, gamma=gamma)
svm_classifier.fit(x_train_tfidf,full_train_data['sentiment'])
svm_predictions = svm_classifier.predict(x_test_tfidf)


# Estimate and print the accuracy and phi coefficient for the random forest classifier
svm_accuracy = accuracy_score(svm_predictions, full_test_data['sentiment'])
svm_phi = matthews_corrcoef(svm_predictions, full_test_data['sentiment'])

print(f"Support Vector Machine accuracy: {svm_accuracy:.2%}")
print(f"Support Vector Machine phi coefficient (correlation): {svm_phi:.02}")
print("\nReady for next step...")

### Gradient Boosting
---


Gradient boosting is an ensemble technique that combines multiple weak learners (e.g., decision trees) similar to the random forest approach we explored above. However, unlike a random forest which looks at individual trees in parallel, gradient boosting constructs trees sequentially, where each new tree aims to address the errors of the previous ones.

In [ ]:
# ~~~~~HYPERPARAMETERS~~~~~
n_estimators = 1000 # Number of 'boosting' rounds
max_depth = 2 # Maximum tree depth
max_leaves = 0 # Maximum number of leaves (set to zero for no limit)
learning_rate = .01 # Boosting learning rate (eta)
objective = "binary:logistic"

# Train the gradient boosting classifier
xgb_classifier = xgb.XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, objective=objective, n_jobs=-1)
xgb_classifier.fit(x_train_tfidf,full_train_data['sentiment'], eval_set=[(x_validation_tfidf, full_validation_data['sentiment'])], verbose=0)
xgb_predictions = xgb_classifier.predict(x_test_tfidf)


# Estimate and print the accuracy and phi coefficient for the gradient boosting classifier
xgb_accuracy = accuracy_score(xgb_predictions, full_test_data['sentiment'])
xgb_phi = matthews_corrcoef(xgb_predictions, full_test_data['sentiment'])

print(f"Gradient Boosting accuracy: {xgb_accuracy:.2%}")
print(f"Gradient Boosting phi coefficient (correlation): {xgb_phi:.02}")
print("\nReady for next step...")

### ACTIVITY
---

In the previous sections we used more-or-less out-of-the-box numbers for the hyperparameters of the machine learning classifiers. Much like other applications of machine learning, you can generally obtain better sentiment classification accuracy by toying around with the parameters a bit.

In the sections above, tweak the **hyperparameter** values and see if you can find models that outperform the base model provided.

### Hyperparameter search and tuning
---

In the above activity, you manually tweaked the machine learning algorithms' hyperparameters to find the best performing model. In practice, however, we generally let the computer do some of this exploration. There are many approaches for doing so, one of these is to do a 'grid search' where you provide the parameters to test and the computer will examine all permutations of these.

You will likely see Convergence warnings, ignore them for the time being - those are models that did not converge in under 1000 iterations

In [ ]:
# ~~~~~HYPERPARAMETERS~~~~~
params = {
    'penalty': ["l2", "l1"],   
    'C': np.logspace(-3,3),
    'solver': ["lbfgs", "liblinear"]
}

# Train the logistic regression classifier with grid search
grid_classifier = linear_model.LogisticRegression(max_iter=1000)
grid = GridSearchCV(estimator=grid_classifier, param_grid=params, scoring='roc_auc', n_jobs=-1, verbose=0)
grid.fit(x_train_tfidf,full_train_data['sentiment'])

print(f"\nBest estimator: {grid.best_estimator_}")
grid_predictions = grid.best_estimator_.predict(x_test_tfidf)

# Estimate and print the accuracy and phi coefficient for the best LR classifier from the grid search
grid_accuracy = accuracy_score(grid_predictions, full_test_data['sentiment'])
grid_phi = matthews_corrcoef(grid_predictions, full_test_data['sentiment'])

print(f"\nGrid Search LR accuracy: {grid_accuracy:.2%}")
print(f"Grid Search LR phi coefficient (correlation): {grid_phi:.02}")
print("\nReady for next step...")

Another consideration when identifying hyperparameters is cross-validation. In k-fold cross-validation, the data is divided into *k* subsamples or "folds". The model training and validation process is then repeated k times, with each of the k folds serving as the validation dataset once, and the remaining k-1 folds comprise the training set. This approach ensures that every data point gets to be in the validation set exactly once and in the training set k-1 times, providing a thorough way of assessing the model's performance.

In [ ]:
# ~~~~~HYPERPARAMETERS~~~~~
params = {
    'penalty': ["l2", "l1"],   
    'C': np.logspace(-3,3),
    'solver': ["lbfgs", "liblinear"]
}
k_folds = 5

# Train the logistic regression classifier
kgrid_classifier = linear_model.LogisticRegression(max_iter=1000)
kgrid = GridSearchCV(estimator=kgrid_classifier, param_grid=params, scoring='roc_auc', cv=k_folds, n_jobs=-1, verbose=0)
kgrid.fit(x_train_tfidf,full_train_data['sentiment'])
print(f"\nBest estimator: {kgrid.best_estimator_}")

kgrid_predictions = kgrid.best_estimator_.predict(x_test_tfidf)

# Estimate and print the accuracy and phi coefficient for the best LR classifier
kgrid_accuracy = accuracy_score(kgrid_predictions, full_test_data['sentiment'])
kgrid_phi = matthews_corrcoef(kgrid_predictions, full_test_data['sentiment'])

print(f"\nGrid Search LR accuracy: {kgrid_accuracy:.2%}")
print(f"Grid Search LR phi coefficient (correlation): {kgrid_phi:.02}")
print("\nReady for next step...")

### A Final Comparison
---

This wraps up the comparison of the sentiment analysis machine learning classification algorithms. As I hope you've seen, this is not a one-size-fits-all decision, there are a number of factors that should enter into your calculus for not only deciding what tool to use, but how to tune and use it.

That being said, the below code will consolidate all of the statistics for the models we ran (net of any tinkering you may have done as part of the activities, of course).

In [ ]:
# Comparison of each sentiment approach
print("CORRELATIONS:")
print(f"The correlation between sentiment and the positivity dictionary is......... ORIGINAL: {pos_pbsr[0]:.02}; p = {pos_pbsr[1]:.02e} --- CUSTOM: {cus_pos_pbsr[0]:.02}; p = {cus_pos_pbsr[1]:.02e}")
print(f"The correlation between sentiment and the negativity dictionary is......... ORIGINAL: {neg_pbsr[0]:.02}; p = {neg_pbsr[1]:.02e} --- CUSTOM:{cus_neg_pbsr[0]:.02}; p = {cus_neg_pbsr[1]:.02e}")
print(f"The correlation between sentiment and the difference score is.............. ORIGINAL: {sent_pbsr[0]:.02}; p = {sent_pbsr[1]:.02e} --- CUSTOM: {cus_sent_pbsr[0]:.02}; p = {cus_sent_pbsr[1]:.02e}")
print(f"The correlation between sentiment and the coefficient of imbalance is...... ORIGINAL: {coi_pbsr[0]:.02}; p = {coi_pbsr[1]:.02e} --- CUSTOM: {cus_coi_pbsr[0]:.02}; p = {cus_coi_pbsr[1]:.02e}")
print(f"The Logistic Regression phi coefficient (correlation) with sentiment is.............. {lr_phi:.02}")
print(f"The Naive Bayes' phi coefficient (correlation) with sentiment is..................... {nb_phi:.02}")
print(f"The Random Forest phi coefficient (correlation) with sentiment is.................... {rf_phi:.02}")
print(f"The Support Vector Machine phi coefficient (correlation) with sentiment is........... {svm_phi:.02}")
print(f"The Gradient Boosting phi coefficient (correlation) with sentiment is................ {xgb_phi:.02}")
print(f"The Grid Search Logistic Regression phi coefficient (correlation) with sentiment is.. {grid_phi:.02}")
print(f"The K-Fold CV Logistic Regression phi coefficient (correlation) with sentiment is.... {kgrid_phi:.02}")



print(f"{'-'*120}")
print("ACCURACIES:")
print(f"Logistic Regression accuracy................... {lr_accuracy:.2%}")
print(f"Naive Bayes' accuracy.......................... {nb_accuracy:.2%}")
print(f"Random Forest accuracy......................... {rf_accuracy:.2%}")
print(f"Support Vector Machine accuracy................ {svm_accuracy:.2%}")
print(f"The Gradient Boosting accuracy................. {xgb_accuracy:.2%}")
print(f"The Grid Search Logistic Regression accuracy... {grid_accuracy:.2%}")
print(f"The K-Fold CV Logistic Regression accuracy..... {kgrid_accuracy:.2%}")

# Unsupervised Machine Learning
---

With supervised machine learning we provided the algorithm with both the inputs and desired output. Specifically, we used a 'classification' algorithm to use text inputs to predict a categorical output (i.e., sentiment). In unsupervised machine learning algorithms, we provide the algorithm with the inputs, but we do not provide it with a desired output.

To provide an analogy to statistical techniques we use in academia:
* Supervised machine learning is like OLS, logistic regression, and SEM - We provide both X and Y variables and the algorithm figures out the best way to link them based on predetermined rules
* Unsupervised machine learning is like exploratory factor analysis, principal components analysis, and cluster analysis - We provide only X variables and the algorithm determines how to combine those X variables in ways that help us understand more about those variables.

## Topic Modeling

There are many unsupervised machine learning algorithms, but a popular one in management research right now is topic modeling.

Topic models are designed to discover the "topics" that occur in a corpus of documents. These models assume that documents are mixtures of topics, which themselves are characterized as a distribution over words. To provide a bit of an oversimplification, but one which academics might find accessible, topic models are a bit like a factor analysis of words. Words that 'hang together' frequently become associated with a latent "topic".

### Latent Dirichlet Allocation
---

Latent Dirichlet Allocation (LDA) is a foundational topic modeling technique used in Natural Language Processing. When applying LDA, a researcher specifies the number of topics to be extracted from the corpus *a priori* (k), and the algorithm then iteratively learns the word distributions for each topic and the topic distributions for each document. 

The below code uses LDA to uncover topics associated with the IMDB dataset we have been working with.

In [ ]:
# ~~~~~HYPERPARAMETERS~~~~~
k = 10 # the number of topics
term_weight = tp.TermWeight.ONE # Term weighting scheme: tp.TermWeight.ONE | tp.TermWeight.PMI | tp.TermWeight.IDF
min_cf = 3 # Minimum corpus frequency (throw out words that do not appear at least this many times in the corpus)
min_df = 1 # Minimum document frequency (throw out words that do not appear at least this many times in a document)
rm_top = 5 # Remove the top N words in terms of frequency
alpha = .1 # hyperparameter of Dirichlet distribution for document-topic
eta = .01 # hyperparameter of Dirichlet distribution for topic-word

# Train Hierarchical Dirichlet Process
model = tp.LDAModel(tw=term_weight, min_cf=min_cf, min_df=min_df, rm_top=rm_top, alpha=alpha, eta=eta, k=k)
for review_text in test_data['review_tokens']:
    model.add_doc(review_text)
model.burn_in=100
model.train(0)
print('Number of documents:', len(model.docs), ', Vocab size:', len(model.used_vocabs), ', Number of words:', model.num_words)
print('Removed top words:', model.removed_top_words)
print('Training...', file=sys.stderr, flush=True)
model.train(1000)

topic_term_dists = np.stack([model.get_topic_word_dist(k) for k in range(model.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in model.docs])
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)
doc_lengths = np.array([len(doc.words) for doc in model.docs])
vocab = list(model.used_vocabs)
term_frequency = model.used_vocab_freq

for k in range(model.k):
    print(f"Topic #{k}")
    for word, prob in model.get_topic_words(k):
        print('\t', word, prob, sep='\t')

In [ ]:
%matplotlib inline
prepared_data = pyLDAvis.prepare(
    topic_term_dists, 
    doc_topic_dists, 
    doc_lengths, 
    vocab, 
    term_frequency,
    start_index=0,
    sort_topics=False
)
pyLDAvis.display(prepared_data)

### Hierarchical Dirichlet Process
---

With Latent Dirichlet Allocation, the researcher specifies *a priori* the number of topics to be found in the corpus of documents. Unfortunately, there is often no *a priori* theory regarding this number. And if there is such theory, there's no way to ensure that those topics are the ones that will emerge in the data. An alternative algorithm that lets the data determine the number of topics is the Hierarchical Dirichlet Process (HDP) algorithm.

In [ ]:
# ~~~~~HYPERPARAMETERS~~~~~
term_weight = tp.TermWeight.IDF # Term weighting scheme: tp.TermWeight.ONE | tp.TermWeight.PMI | tp.TermWeight.IDF
min_cf = 3 # Minimum corpus frequency (throw out words that do not appear at least this many times in the corpus)
min_df = 1 # Minimum document frequency (throw out words that do not appear at least this many times in a document)
rm_top = 5 # Remove the top N words in terms of frequency
alpha = 0.1 # concentration coefficient of Dirichlet Process for document-table
eta = 0.01 # hyperparameter of Dirichlet distribution for topic-word
gamma = 0.1 # concentration coeficient of Dirichlet Process for table-topic
initial_k = 10

# Train Hierarchical Dirichlet Process
model = tp.HDPModel(tw=term_weight, min_cf=min_cf, min_df=min_df, rm_top=rm_top, alpha=alpha, eta=eta, gamma=gamma, initial_k=initial_k)
for review_text in test_data['review_tokens']:
    model.add_doc(review_text)
model.burn_in=100
model.train(0)
print('Number of documents:', len(model.docs), ', Vocab size:', len(model.used_vocabs), ', Number of words:', model.num_words)
print('Removed top words:', model.removed_top_words)
print('Training...', flush=True)
model.train(1000)

live_topics = [k for k in range(model.k) if model.is_live_topic(k)]

topic_term_dists = np.stack([model.get_topic_word_dist(k) for k in range(model.k)])
topic_term_dists = topic_term_dists[live_topics]
topic_term_dists /= topic_term_dists.sum(axis=1, keepdims=True)

doc_topic_dists = np.stack([doc.get_topic_dist() for doc in model.docs])
doc_topic_dists = doc_topic_dists[:, live_topics]
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)

doc_lengths = np.array([len(doc.words) for doc in model.docs])
vocab = list(model.used_vocabs)
term_frequency = model.used_vocab_freq

In [ ]:
%matplotlib inline
prepared_data = pyLDAvis.prepare(
    topic_term_dists, 
    doc_topic_dists, 
    doc_lengths, 
    vocab, 
    term_frequency,
    start_index=0,
    sort_topics=False
)
pyLDAvis.display(prepared_data)

### ACTIVITY
---

In the previous section, I provided out-of-the-box numbers for the hyperparameters of the topic models. Tweaking these hyperparameters can influence the interpretability of the topics that emerge. In the sections above, tweak the **hyperparameter** values and see if you can find models that produce interesting/insightful topics regarding the movie reviews.